In [ ]:
!git clone https://github.com/ahmedalashii/Yolov11-Ingredients-Detection.git

Cloning into 'Yolov11-Ingredients-Detection'...
remote: Enumerating objects: 4139, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (52/52), done.
remote: Total 4139 (delta 3), reused 53 (delta 3), pack-reused 4084 (from 2)
Receiving objects: 100% (4139/4139), 290.58 MiB | 38.87 MiB/s, done.
Resolving deltas: 100% (21/21), done.
Updating files: 100% (4141/4141), done.


In [ ]:
%cd Yolov11-Ingredients-Detection

/content/Yolov11-Ingredients-Detection


In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 95.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 91.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 43.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

In [ ]:
from ultralytics import YOLO
import os

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
ROOT_DIR = os.getcwd()

# Dataset YAML path
SINGLE_INGREDIENT_DATA_YAML_PATH = os.path.join(ROOT_DIR, 'datasets', 'yolo_single_ingredient_dataset', 'data.yaml')


# Pretrained model path
ORIGINAL_YOLO_MODEL_PATH = os.path.join(ROOT_DIR, 'models', 'original_yolo11s.pt')

# Output dir for saving trained models
OUTPUT_DIR = os.path.join(ROOT_DIR, 'models')
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [ ]:
import yaml

with open(SINGLE_INGREDIENT_DATA_YAML_PATH, 'r') as file:
    class_names = yaml.safe_load(file)['names']
print("Class names loaded from YAML file:", class_names)
print(f"Number of classes: {len(class_names)}")

Class names loaded from YAML file: ['banana', 'bell pepper', 'bulgur', 'cabbage', 'carrot', 'cheese', 'chicken', 'chickpeas', 'cucumber', 'egg', 'eggplant', 'farfalle pasta', 'fusilli pasta', 'garlic', 'green pepper', 'ground meat', 'ketchup', 'lemon', 'margarine', 'mayonnaise', 'milk', 'mushroom', 'olive', 'olive oil', 'onion', 'parsley', 'potato', 'red lentils', 'rice', 'sausage', 'spaghetti', 'tomato', 'tomato paste', 'turkish dumplings', 'yogurt', 'zucchini']
Number of classes: 36


In [ ]:
import yaml
import os

# Load the data.yaml file
with open(SINGLE_INGREDIENT_DATA_YAML_PATH, 'r') as file:
    data = yaml.safe_load(file)

class_names = data['names']
num_classes = len(class_names)

# Function to count instances per class in a given directory
def count_classes(label_dir, class_names):
    class_counts = {name: 0 for name in class_names}
    if not os.path.exists(label_dir):
        print(f"Warning: Directory not found: {label_dir}")
        return class_counts

    for label_file in os.listdir(label_dir):
        if label_file.endswith('.txt'):
            with open(os.path.join(label_dir, label_file), 'r') as f:
                for line in f:
                    try:
                        class_id = int(line.split()[0])
                        if 0 <= class_id < len(class_names):
                            class_name = class_names[class_id]
                            class_counts[class_name] += 1
                        else:
                            print(f"Warning: Invalid class ID {class_id} in file {label_file}")
                    except (ValueError, IndexError):
                        print(f"Warning: Could not parse line in file {label_file}: {line.strip()}")
    return class_counts

# Define paths to label directories for each split
train_label_dir = os.path.join(ROOT_DIR, 'datasets', 'yolo_single_ingredient_dataset', 'train', 'labels')
val_label_dir = os.path.join(ROOT_DIR, 'datasets', 'yolo_single_ingredient_dataset', 'valid', 'labels')
test_label_dir = os.path.join(ROOT_DIR, 'datasets', 'yolo_single_ingredient_dataset', 'test', 'labels')


# Get the path to the training, validation, and test images
train_img_dir = os.path.join(ROOT_DIR, 'datasets', 'yolo_single_ingredient_dataset', 'train', 'images')
val_img_dir = os.path.join(ROOT_DIR, 'datasets', 'yolo_single_ingredient_dataset', 'valid', 'images')
test_img_dir = os.path.join(ROOT_DIR, 'datasets', 'yolo_single_ingredient_dataset', 'test', 'images')


# Count instances for each split
train_class_counts = count_classes(train_label_dir, class_names)
val_class_counts = count_classes(val_label_dir, class_names)
test_class_counts = count_classes(test_label_dir, class_names)

# Calculate overall counts
overall_class_counts = {name: train_class_counts[name] + val_class_counts[name] + test_class_counts[name] for name in class_names}


# Print results
print("Class Distribution across Splits:")
print("\nTraining Set:")
print("Number of images:", len(os.listdir(train_img_dir)))
print("Number of instances per class:")
for class_name, count in train_class_counts.items():
    print(f"{class_name:<20}: {count}")

print("\nValidation Set:")
print("Number of images:", len(os.listdir(val_img_dir)))
print("Number of instances per class:")
for class_name, count in val_class_counts.items():
    print(f"{class_name:<20}: {count}")

print("\nTest Set:")
print("Number of images:", len(os.listdir(test_img_dir)))
print("Number of instances per class:")
for class_name, count in test_class_counts.items():
    print(f"{class_name:<20}: {count}")


print("\nOverall Dataset:")
print("Number of images:", len(os.listdir(train_img_dir)) + len(os.listdir(val_img_dir)) + len(os.listdir(test_img_dir)))
print("Number of instances per class:")
for class_name, count in overall_class_counts.items():
    print(f"{class_name:<20}: {count}")

Class Distribution across Splits:

Training Set:
Number of images: 1425
Number of instances per class:
banana              : 87
bell pepper         : 151
bulgur              : 15
cabbage             : 11
carrot              : 16
cheese              : 63
chicken             : 20
chickpeas           : 32
cucumber            : 118
egg                 : 254
eggplant            : 81
farfalle pasta      : 18
fusilli pasta       : 18
garlic              : 126
green pepper        : 122
ground meat         : 18
ketchup             : 20
lemon               : 351
margarine           : 24
mayonnaise          : 21
milk                : 99
mushroom            : 261
olive               : 189
olive oil           : 69
onion               : 264
parsley             : 19
potato              : 360
red lentils         : 16
rice                : 76
sausage             : 16
spaghetti           : 20
tomato              : 325
tomato paste        : 80
turkish dumplings   : 13
yogurt              : 28
zucchini   

In [ ]:
import pandas as pd

# Create a DataFrame from the class counts
df_class_counts = pd.DataFrame({
    'Class Name': class_names,
    'Training Set': train_class_counts.values(),
    'Validation Set': val_class_counts.values(),
    'Test Set': test_class_counts.values(),
    'Overall Dataset': overall_class_counts.values()
})

# Display the DataFrame
print("Class Distribution across Splits:")
display(df_class_counts)

Class Distribution across Splits:


,Class Name,Training Set,Validation Set,Test Set,Overall Dataset
0,banana,87,12,11,110
1,bell pepper,151,26,35,212
2,bulgur,15,1,2,18
3,cabbage,11,2,3,16
4,carrot,16,5,2,23
5,cheese,63,21,7,91
6,chicken,20,13,1,34
7,chickpeas,32,4,4,40
8,cucumber,118,26,17,161
9,egg,254,37,52,343


In [ ]:
# Original YOLO Evaluation on single-ingredient images

from ultralytics import YOLO
import numpy as np
import os

# Define paths
YOLO_MODEL_PATH = os.path.join(ROOT_DIR, 'models', 'original_yolo11s.pt')

# Load the original model
original_model = YOLO(YOLO_MODEL_PATH)

# Evaluate the model
metrics = original_model.val(data=SINGLE_INGREDIENT_DATA_YAML_PATH)

# Extract metrics
map50 = metrics.box.map50         # mAP@0.5
map50_95 = metrics.box.map        # mAP@0.5:0.95
precision_all = metrics.box.p     # list of precision per class
recall_all = metrics.box.r        # list of recall per class
class_names = original_model.names # class ID to name mapping

# Print mean metrics
print(f"\nOverall Metrics:")
print(f"Mean Precision: {np.mean(precision_all):.4f}")
print(f"Mean Recall: {np.mean(recall_all):.4f}")
print(f"mAP@0.5: {map50:.4f}")
print(f"mAP@0.5:0.95: {map50_95:.4f}")

# Print per-class precision and recall
print("\nPer-Class Metrics:")
for idx, (prec, rec) in enumerate(zip(precision_all, recall_all)):
    class_name = class_names[idx] if class_names and idx < len(class_names) else f"Class {idx}"
    print(f"Class ID {idx:2d} ({class_name:<20}): Precision = {prec:.4f}, Recall = {rec:.4f}")

Ultralytics 8.3.174 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11s summary (fused): 100 layers, 9,443,760 parameters, 0 gradients, 21.5 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1537.0±458.5 MB/s, size: 50.9 KB)


val: Scanning /content/Yolov11-Ingredients-Detection/datasets/yolo_single_ingredient_dataset/valid/labels.cache... 286 images, 0 backgrounds, 0 corrupt: 100%|██████████| 286/286 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [00:05<00:00,  3.44it/s]


                   all        286        728    0.00751     0.0849    0.00807    0.00488
                person          8         12    0.00156     0.0833    0.00088   0.000352
               bicycle          6         26          0          0          0          0
                   car          1          1          0          0          0          0
            motorcycle          1          2          0          0          0          0
              airplane          3          5          0          0          0          0
                   bus         15         21     0.0408     0.0952     0.0234     0.0166
                 train          3         13          0          0          0          0
                 truck          3          4          0          0          0          0
                  boat         10         26     0.0179     0.0769     0.0112    0.00853
         traffic light         13         37          0          0          0          0
          fire hydran

In [ ]:
# Original YOLO Evaluation on multiple-ingredient images

from ultralytics import YOLO
import numpy as np
import os

# Define paths
YOLO_MODEL_PATH = os.path.join(ROOT_DIR, 'models', 'original_yolo11s.pt')

# Load the original model
original_model = YOLO(YOLO_MODEL_PATH)

# Evaluate the model
metrics = original_model.val(data=MULTIPLE_INGREDIENTS_DATA_YAML_PATH)

# Extract metrics
map50 = metrics.box.map50         # mAP@0.5
map50_95 = metrics.box.map        # mAP@0.5:0.95
precision_all = metrics.box.p     # list of precision per class
recall_all = metrics.box.r        # list of recall per class
class_names = original_model.names # class ID to name mapping

# Print mean metrics
print(f"\nOverall Metrics:")
print(f"Mean Precision: {np.mean(precision_all):.4f}")
print(f"Mean Recall: {np.mean(recall_all):.4f}")
print(f"mAP@0.5: {map50:.4f}")
print(f"mAP@0.5:0.95: {map50_95:.4f}")

# Print per-class precision and recall
print("\nPer-Class Metrics:")
for idx, (prec, rec) in enumerate(zip(precision_all, recall_all)):
    class_name = class_names[idx] if class_names and idx < len(class_names) else f"Class {idx}"
    print(f"Class ID {idx:2d} ({class_name:<20}): Precision = {prec:.4f}, Recall = {rec:.4f}")

Ultralytics 8.3.174 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11s summary (fused): 100 layers, 9,443,760 parameters, 0 gradients, 21.5 GFLOPs


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 340.4±398.0 MB/s, size: 106.6 KB)



val: Scanning /content/Yolov11-Ingredients-Detection/datasets/yolo_multi_ingredient_dataset/valid/labels... 53 images, 0 backgrounds, 0 corrupt: 100%|██████████| 53/53 [00:00<00:00, 126.23it/s]

val: New cache created: /content/Yolov11-Ingredients-Detection/datasets/yolo_multi_ingredient_dataset/valid/labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:03<00:00,  1.22it/s]


                   all         53        169      0.037     0.0208     0.0266     0.0151
                person          5          8          0          0          0          0
               bicycle          2          4          0          0          0          0
                   car          1          2          0          0          0          0
            motorcycle          2          3          0          0          0          0
              airplane          2          2          0          0          0          0
                   bus          3          4          1       0.25      0.625      0.312
                 train          2          2          0          0          0          0
                 truck          2          2          0          0          0          0
                  boat          3          3          0          0          0          0
         traffic light          6         15          0          0          0          0
          fire hydran

In [ ]:
# model_30 = YOLO(ORIGINAL_YOLO_MODEL_PATH)
# model_30.train(data=SINGLE_INGREDIENT_DATA_YAML_PATH, epochs=30, imgsz=640, project=OUTPUT_DIR, name='yolo11s_30_epochs')

Ultralytics 8.3.174 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/Yolov11-Thesis-Datasets/datasets/yolo_single_ingredient_dataset/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/content/Yolov11-Thesis-Datasets/models/original_yolo11s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolo11s_30_epochs, nbs=64, nms=False, opset=None, 

Overriding model.yaml nc=80 with nc=36

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  3                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  4                  -1  1    103360  ultralytics.nn.modules.block.C3k2            [128, 256, 1, False, 0.25]    
  5                  -1  1    590336  ultralytics.nn.modules.conv.Conv             [256, 256, 3, 2]              
  6                  -1  1    346112  ultralytics.nn.modules.block.C3k2            [256, 256, 1, True]           
  7                  -1  1   1180672  ultralytic

  8                  -1  1   1380352  ultralytics.nn.modules.block.C3k2            [512, 512, 1, True]           
  9                  -1  1    656896  ultralytics.nn.modules.block.SPPF            [512, 512, 5]                 
 10                  -1  1    990976  ultralytics.nn.modules.block.C2PSA           [512, 512, 1]                 
 11                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 12             [-1, 6]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 13                  -1  1    443776  ultralytics.nn.modules.block.C3k2            [768, 256, 1, False]          
 14                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 15             [-1, 4]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 16                  -1  1    127680  ultralytics.nn.modules.block.C3k2            [512,

AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 26.9±8.9 MB/s, size: 68.6 KB)


train: Scanning /content/Yolov11-Thesis-Datasets/datasets/yolo_single_ingredient_dataset/train/labels... 1425 images, 1 backgrounds, 0 corrupt: 100%|██████████| 1425/1425 [00:02<00:00, 661.62it/s]


...

                   all        286        728      0.845      0.761      0.863      0.651



30 epochs completed in 0.269 hours.
Optimizer stripped from /content/Yolov11-Thesis-Datasets/models/trained_models/yolo11s_30_epochs/weights/last.pt, 19.2MB
Optimizer stripped from /content/Yolov11-Thesis-Datasets/models/trained_models/yolo11s_30_epochs/weights/best.pt, 19.2MB

Validating /content/Yolov11-Thesis-Datasets/models/trained_models/yolo11s_30_epochs/weights/best.pt...
Ultralytics 8.3.174 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11s summary (fused): 100 layers, 9,426,732 parameters, 0 gradients, 21.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:04<00:00,  1.94it/s]


                   all        286        728      0.775      0.774      0.852      0.658
                banana          8         12      0.763      0.667      0.802      0.536
           bell pepper          6         26      0.725      0.885      0.916      0.701
                bulgur          1          1       0.46          1      0.995      0.697
               cabbage          1          2          1      0.739      0.995      0.796
                carrot          3          5      0.777      0.709      0.928      0.562
                cheese         15         21      0.873      0.571       0.78      0.548
               chicken          3         13       0.98          1      0.995      0.802
             chickpeas          3          4      0.893          1      0.995      0.635
              cucumber         10         26      0.838      0.797      0.866      0.661
                   egg         13         37      0.773      0.784      0.869      0.712
              eggplan

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7fab6271ed10>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038, 

In [ ]:
import shutil
import os

# Path to the training output directory
trained_model_dir = os.path.join(OUTPUT_DIR, 'yolo11s_30_epochs')

# Output zip file path
zip_output_path = os.path.join(OUTPUT_DIR, 'yolo11s_30_epochs.zip')

# Zip the folder
shutil.make_archive(base_name=zip_output_path.replace('.zip', ''), format='zip', root_dir=trained_model_dir)

print(f"Zipped to: {zip_output_path}")

Zipped to: /content/Yolov11-Thesis-Datasets/models/trained_models/yolo11s_30_epochs.zip


In [ ]:
# 30 epochs Fine-tuned YOLO Evaluation on single-ingredient images

from ultralytics import YOLO
import numpy as np
import os

# Define paths
YOLO_MODEL_30_PATH = os.path.join(ROOT_DIR, 'models', 'yolo11s_30_epochs', 'weights', 'best.pt')

# Load the trained model
trained_model = YOLO(YOLO_MODEL_30_PATH)

# Evaluate the model
metrics = trained_model.val(data=SINGLE_INGREDIENT_DATA_YAML_PATH)

# Extract metrics
map50 = metrics.box.map50         # mAP@0.5
map50_95 = metrics.box.map        # mAP@0.5:0.95
precision_all = metrics.box.p     # list of precision per class
recall_all = metrics.box.r        # list of recall per class
class_names = trained_model.names # class ID to name mapping

# Print mean metrics
print(f"\nOverall Metrics:")
print(f"Mean Precision: {np.mean(precision_all):.4f}")
print(f"Mean Recall: {np.mean(recall_all):.4f}")
print(f"mAP@0.5: {map50:.4f}")
print(f"mAP@0.5:0.95: {map50_95:.4f}")

# Print per-class precision and recall
print("\nPer-Class Metrics:")
for idx, (prec, rec) in enumerate(zip(precision_all, recall_all)):
    class_name = class_names[idx] if class_names and idx in class_names else f"Class {idx}"
    print(f"Class ID {idx:2d} ({class_name:<20}): Precision = {prec:.4f}, Recall = {rec:.4f}")

Ultralytics 8.3.174 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11s summary (fused): 100 layers, 9,426,732 parameters, 0 gradients, 21.4 GFLOPs


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 30.2±10.9 MB/s, size: 55.3 KB)



val: Scanning /content/Yolov11-Ingredients-Detection/datasets/yolo_single_ingredient_dataset/valid/labels... 286 images, 0 backgrounds, 0 corrupt: 100%|██████████| 286/286 [00:00<00:00, 655.81it/s]

val: New cache created: /content/Yolov11-Ingredients-Detection/datasets/yolo_single_ingredient_dataset/valid/labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [00:07<00:00,  2.48it/s]


                   all        286        728      0.769      0.778      0.851      0.658
                banana          8         12      0.756      0.667      0.802      0.535
           bell pepper          6         26       0.72      0.885      0.916      0.701
                bulgur          1          1      0.452          1      0.995      0.697
               cabbage          1          2          1      0.761      0.995      0.796
                carrot          3          5      0.779      0.715      0.928      0.562
                cheese         15         21      0.808      0.571       0.76      0.535
               chicken          3         13      0.977          1      0.995      0.802
             chickpeas          3          4      0.888          1      0.995      0.648
              cucumber         10         26       0.84      0.808      0.866      0.662
                   egg         13         37      0.768      0.784      0.869       0.71
              eggplan

In [ ]:
# 30 epochs Fine-tuned YOLO Evaluation on multiple-ingredient images

from ultralytics import YOLO
import numpy as np
import os

# Define paths
YOLO_MODEL_30_PATH = os.path.join(ROOT_DIR, 'models', 'yolo11s_30_epochs', 'weights', 'best.pt')

# Load the trained model
trained_model = YOLO(YOLO_MODEL_30_PATH)

# Evaluate the model
metrics = trained_model.val(data=MULTIPLE_INGREDIENTS_DATA_YAML_PATH)

# Extract metrics
map50 = metrics.box.map50         # mAP@0.5
map50_95 = metrics.box.map        # mAP@0.5:0.95
precision_all = metrics.box.p     # list of precision per class
recall_all = metrics.box.r        # list of recall per class
class_names = trained_model.names # class ID to name mapping

# Print mean metrics
print(f"\nOverall Metrics:")
print(f"Mean Precision: {np.mean(precision_all):.4f}")
print(f"Mean Recall: {np.mean(recall_all):.4f}")
print(f"mAP@0.5: {map50:.4f}")
print(f"mAP@0.5:0.95: {map50_95:.4f}")

# Print per-class precision and recall
print("\nPer-Class Metrics:")
for idx, (prec, rec) in enumerate(zip(precision_all, recall_all)):
    class_name = class_names[idx] if class_names and idx in class_names else f"Class {idx}"
    print(f"Class ID {idx:2d} ({class_name:<20}): Precision = {prec:.4f}, Recall = {rec:.4f}")

Ultralytics 8.3.174 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11s summary (fused): 100 layers, 9,426,732 parameters, 0 gradients, 21.4 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1573.2±1215.1 MB/s, size: 1018.1 KB)


val: Scanning /content/Yolov11-Ingredients-Detection/datasets/yolo_multi_ingredient_dataset/valid/labels.cache... 53 images, 0 backgrounds, 0 corrupt: 100%|██████████| 53/53 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:02<00:00,  1.77it/s]


                   all         53        169      0.565      0.423      0.451      0.324
                banana          5          8      0.907      0.875      0.971      0.672
           bell pepper          2          4      0.362       0.25      0.273      0.206
                bulgur          1          2          1          0          0          0
               cabbage          2          3          1          0          0          0
                carrot          2          2          1          0      0.508      0.407
                cheese          3          4      0.163        0.5      0.499      0.279
               chicken          2          2      0.287        0.5       0.29      0.191
             chickpeas          2          2      0.381        0.5      0.745      0.348
              cucumber          3          3      0.458      0.583      0.358      0.262
                   egg          6         15          1      0.327      0.805      0.523
              eggplan

In [ ]:
# Only if there's a need for training over 60 epochs
# model_60 = YOLO(ORIGINAL_YOLO_MODEL_PATH)
# model_60.train(data=SINGLE_INGREDIENT_DATA_YAML_PATH, epochs=60, imgsz=640, project=OUTPUT_DIR, name='yolo11s_60_epochs')

Ultralytics 8.3.174 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/Yolov11-Thesis-Datasets/datasets/yolo_single_ingredient_dataset/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=60, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/content/Yolov11-Thesis-Datasets/models/original_yolo11s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolo11s_60_epochs, nbs=64, nms=False, opset=None, 

train: Scanning /content/Yolov11-Thesis-Datasets/datasets/yolo_single_ingredient_dataset/train/labels.cache... 1425 images, 1 backgrounds, 0 corrupt: 100%|██████████| 1425/1425 [00:00<?, ?it/s]

WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 14, len(boxes) = 3542. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 516.6±247.9 MB/s, size: 56.8 KB)


val: Scanning /content/Yolov11-Thesis-Datasets/datasets/yolo_single_ingredient_dataset/valid/labels.cache... 286 images, 0 backgrounds, 0 corrupt: 100%|██████████| 286/286 [00:00<?, ?it/s]


...

                   all        286        728      0.811      0.816      0.866      0.658



60 epochs completed in 0.537 hours.
Optimizer stripped from /content/Yolov11-Thesis-Datasets/models/trained_models/yolo11s_60_epochs/weights/last.pt, 19.2MB
Optimizer stripped from /content/Yolov11-Thesis-Datasets/models/trained_models/yolo11s_60_epochs/weights/best.pt, 19.2MB

Validating /content/Yolov11-Thesis-Datasets/models/trained_models/yolo11s_60_epochs/weights/best.pt...
Ultralytics 8.3.174 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11s summary (fused): 100 layers, 9,426,732 parameters, 0 gradients, 21.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:05<00:00,  1.56it/s]


                   all        286        728      0.849      0.765       0.86      0.668
                banana          8         12      0.803       0.75      0.704      0.497
           bell pepper          6         26      0.747      0.923      0.915      0.708
                bulgur          1          1          1          0      0.995      0.796
               cabbage          1          2        0.9          1      0.995      0.847
                carrot          3          5          1      0.469      0.995      0.824
                cheese         15         21      0.906      0.619      0.807      0.558
               chicken          3         13       0.98          1      0.995      0.796
             chickpeas          3          4          1      0.848      0.995      0.575
              cucumber         10         26      0.858      0.697      0.903      0.708
                   egg         13         37      0.885      0.833      0.881      0.719
              eggplan

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7fab628f2850>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038, 

In [ ]:
import shutil
import os

# Path to the training output directory
trained_model_dir = os.path.join(OUTPUT_DIR, 'yolo11s_60_epochs')

# Output zip file path
zip_output_path = os.path.join(OUTPUT_DIR, 'yolo11s_60_epochs.zip')

# Zip the folder
shutil.make_archive(base_name=zip_output_path.replace('.zip', ''), format='zip', root_dir=trained_model_dir)

print(f"Zipped to: {zip_output_path}")

Zipped to: /content/Yolov11-Thesis-Datasets/models/trained_models/yolo11s_60_epochs.zip


In [ ]:
# 60 epochs Fine-tuned YOLO Evaluation on single-ingredient images

from ultralytics import YOLO
import numpy as np
import os

# Define paths
YOLO_MODEL_60_PATH = os.path.join(ROOT_DIR, 'models', 'yolo11s_60_epochs', 'weights', 'best.pt')

# Load the trained model
trained_model = YOLO(YOLO_MODEL_60_PATH)

# Evaluate the model
metrics = trained_model.val(data=SINGLE_INGREDIENT_DATA_YAML_PATH)

# Extract metrics
map50 = metrics.box.map50         # mAP@0.5
map50_95 = metrics.box.map        # mAP@0.5:0.95
precision_all = metrics.box.p     # list of precision per class
recall_all = metrics.box.r        # list of recall per class
class_names = trained_model.names # class ID to name mapping

# Print mean metrics
print(f"\nOverall Metrics:")
print(f"Mean Precision: {np.mean(precision_all):.4f}")
print(f"Mean Recall: {np.mean(recall_all):.4f}")
print(f"mAP@0.5: {map50:.4f}")
print(f"mAP@0.5:0.95: {map50_95:.4f}")

# Print per-class precision and recall
print("\nPer-Class Metrics:")
for idx, (prec, rec) in enumerate(zip(precision_all, recall_all)):
    class_name = class_names[idx] if class_names and idx in class_names else f"Class {idx}"
    print(f"Class ID {idx:2d} ({class_name:<20}): Precision = {prec:.4f}, Recall = {rec:.4f}")

Ultralytics 8.3.174 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11s summary (fused): 100 layers, 9,426,732 parameters, 0 gradients, 21.4 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1260.8±302.7 MB/s, size: 56.4 KB)


val: Scanning /content/Yolov11-Ingredients-Detection/datasets/yolo_single_ingredient_dataset/valid/labels.cache... 286 images, 0 backgrounds, 0 corrupt: 100%|██████████| 286/286 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [00:06<00:00,  2.72it/s]


                   all        286        728      0.848      0.765      0.859      0.668
                banana          8         12      0.802       0.75      0.704      0.497
           bell pepper          6         26      0.747      0.923      0.916      0.709
                bulgur          1          1          1          0      0.995      0.796
               cabbage          1          2        0.9          1      0.995      0.847
                carrot          3          5          1      0.472      0.995      0.824
                cheese         15         21      0.906      0.619       0.81      0.563
               chicken          3         13       0.98          1      0.995        0.8
             chickpeas          3          4          1      0.848      0.995      0.575
              cucumber         10         26      0.858      0.698      0.903      0.701
                   egg         13         37      0.885      0.834       0.88      0.719
              eggplan

In [ ]:
# 60 epochs Fine-tuned YOLO Evaluation on multiple-ingredient images

from ultralytics import YOLO
import numpy as np
import os

# Define paths
YOLO_MODEL_60_PATH = os.path.join(ROOT_DIR, 'models', 'yolo11s_60_epochs', 'weights', 'best.pt')

# Load the trained model
trained_model = YOLO(YOLO_MODEL_60_PATH)

# Evaluate the model
metrics = trained_model.val(data=MULTIPLE_INGREDIENTS_DATA_YAML_PATH)

# Extract metrics
map50 = metrics.box.map50         # mAP@0.5
map50_95 = metrics.box.map        # mAP@0.5:0.95
precision_all = metrics.box.p     # list of precision per class
recall_all = metrics.box.r        # list of recall per class
class_names = trained_model.names # class ID to name mapping

# Print mean metrics
print(f"\nOverall Metrics:")
print(f"Mean Precision: {np.mean(precision_all):.4f}")
print(f"Mean Recall: {np.mean(recall_all):.4f}")
print(f"mAP@0.5: {map50:.4f}")
print(f"mAP@0.5:0.95: {map50_95:.4f}")

# Print per-class precision and recall
print("\nPer-Class Metrics:")
for idx, (prec, rec) in enumerate(zip(precision_all, recall_all)):
    class_name = class_names[idx] if class_names and idx in class_names else f"Class {idx}"
    print(f"Class ID {idx:2d} ({class_name:<20}): Precision = {prec:.4f}, Recall = {rec:.4f}")

Ultralytics 8.3.174 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11s summary (fused): 100 layers, 9,426,732 parameters, 0 gradients, 21.4 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1613.8±632.4 MB/s, size: 84.8 KB)


val: Scanning /content/Yolov11-Ingredients-Detection/datasets/yolo_multi_ingredient_dataset/valid/labels.cache... 53 images, 0 backgrounds, 0 corrupt: 100%|██████████| 53/53 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:03<00:00,  1.14it/s]


                   all         53        169      0.482      0.458      0.446      0.279
                banana          5          8      0.704      0.875      0.783      0.521
           bell pepper          2          4      0.256       0.25      0.239      0.171
                bulgur          1          2          1          0          0          0
               cabbage          2          3          1          0      0.322     0.0567
                carrot          2          2      0.556        0.5      0.523      0.366
                cheese          3          4      0.172       0.25      0.121      0.082
               chicken          2          2       0.64          1      0.828      0.349
             chickpeas          2          2      0.566        0.5      0.828        0.3
              cucumber          3          3      0.689          1       0.83      0.561
                   egg          6         15      0.524        0.6      0.735      0.539
              eggplan

In [ ]:
# Only if there's a need for training over 100 epochs
# model_100 = YOLO(ORIGINAL_YOLO_MODEL_PATH)
# model_100.train(data=SINGLE_INGREDIENT_DATA_YAML_PATH, epochs=100, imgsz=640, project=OUTPUT_DIR, name='yolo11s_100_epochs')

Ultralytics 8.3.174 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/Yolov11-Thesis-Datasets/datasets/yolo_single_ingredient_dataset/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/content/Yolov11-Thesis-Datasets/models/original_yolo11s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolo11s_100_epochs, nbs=64, nms=False, opset=None

train: Scanning /content/Yolov11-Thesis-Datasets/datasets/yolo_single_ingredient_dataset/train/labels.cache... 1425 images, 1 backgrounds, 0 corrupt: 100%|██████████| 1425/1425 [00:00<?, ?it/s]

WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 14, len(boxes) = 3542. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 602.9±460.7 MB/s, size: 56.8 KB)


val: Scanning /content/Yolov11-Thesis-Datasets/datasets/yolo_single_ingredient_dataset/valid/labels.cache... 286 images, 0 backgrounds, 0 corrupt: 100%|██████████| 286/286 [00:00<?, ?it/s]


...

                   all        286        728      0.848      0.827      0.861       0.67



100 epochs completed in 0.929 hours.
Optimizer stripped from /content/Yolov11-Thesis-Datasets/models/trained_models/yolo11s_100_epochs/weights/last.pt, 19.2MB
Optimizer stripped from /content/Yolov11-Thesis-Datasets/models/trained_models/yolo11s_100_epochs/weights/best.pt, 19.2MB

Validating /content/Yolov11-Thesis-Datasets/models/trained_models/yolo11s_100_epochs/weights/best.pt...
Ultralytics 8.3.174 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11s summary (fused): 100 layers, 9,426,732 parameters, 0 gradients, 21.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:08<00:00,  1.10it/s]


                   all        286        728      0.872      0.822      0.871      0.678
                banana          8         12      0.811      0.667      0.745      0.545
           bell pepper          6         26      0.827      0.917      0.905      0.704
                bulgur          1          1      0.953          1      0.995      0.697
               cabbage          1          2      0.946          1      0.995      0.653
                carrot          3          5          1      0.871      0.995      0.826
                cheese         15         21      0.931      0.714      0.871       0.64
               chicken          3         13      0.982          1      0.995      0.783
             chickpeas          3          4      0.961          1      0.995      0.602
              cucumber         10         26      0.949      0.716      0.911      0.688
                   egg         13         37      0.842      0.838       0.85        0.7
              eggplan

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7fab61f173d0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038, 

In [ ]:
import shutil
import os

# Path to the training output directory
trained_model_dir = os.path.join(OUTPUT_DIR, 'yolo11s_100_epochs')

# Output zip file path
zip_output_path = os.path.join(OUTPUT_DIR, 'yolo11s_100_epochs.zip')

# Zip the folder
shutil.make_archive(base_name=zip_output_path.replace('.zip', ''), format='zip', root_dir=trained_model_dir)

print(f"Zipped to: {zip_output_path}")

Zipped to: /content/Yolov11-Thesis-Datasets/models/trained_models/yolo11s_100_epochs.zip


In [ ]:
# 100 epochs Fine-tuned YOLO Evaluation on single-ingredient images

from ultralytics import YOLO
import numpy as np
import os

# Define paths
YOLO_MODEL_100_PATH = os.path.join(ROOT_DIR, 'models', 'yolo11s_100_epochs', 'weights', 'best.pt')

# Load the trained model
trained_model = YOLO(YOLO_MODEL_100_PATH)

# Evaluate the model
metrics = trained_model.val(data=SINGLE_INGREDIENT_DATA_YAML_PATH)

# Extract metrics
map50 = metrics.box.map50         # mAP@0.5
map50_95 = metrics.box.map        # mAP@0.5:0.95
precision_all = metrics.box.p     # list of precision per class
recall_all = metrics.box.r        # list of recall per class
class_names = trained_model.names # class ID to name mapping

# Print mean metrics
print(f"\nOverall Metrics:")
print(f"Mean Precision: {np.mean(precision_all):.4f}")
print(f"Mean Recall: {np.mean(recall_all):.4f}")
print(f"mAP@0.5: {map50:.4f}")
print(f"mAP@0.5:0.95: {map50_95:.4f}")

# Print per-class precision and recall
print("\nPer-Class Metrics:")
for idx, (prec, rec) in enumerate(zip(precision_all, recall_all)):
    class_name = class_names[idx] if class_names and idx in class_names else f"Class {idx}"
    print(f"Class ID {idx:2d} ({class_name:<20}): Precision = {prec:.4f}, Recall = {rec:.4f}")

Ultralytics 8.3.174 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11s summary (fused): 100 layers, 9,426,732 parameters, 0 gradients, 21.4 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1174.6±207.9 MB/s, size: 72.5 KB)


val: Scanning /content/Yolov11-Ingredients-Detection/datasets/yolo_single_ingredient_dataset/valid/labels.cache... 286 images, 0 backgrounds, 0 corrupt: 100%|██████████| 286/286 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [00:05<00:00,  3.31it/s]


                   all        286        728      0.872      0.821      0.871      0.676
                banana          8         12      0.817      0.667      0.744      0.545
           bell pepper          6         26      0.826      0.915      0.903      0.702
                bulgur          1          1      0.956          1      0.995      0.697
               cabbage          1          2      0.948          1      0.995      0.653
                carrot          3          5          1       0.87      0.995      0.826
                cheese         15         21      0.932      0.714      0.871       0.64
               chicken          3         13      0.982          1      0.995      0.783
             chickpeas          3          4      0.962          1      0.995      0.602
              cucumber         10         26      0.919      0.692      0.909      0.696
                   egg         13         37      0.843      0.838       0.85      0.699
              eggplan

In [ ]:
# 100 epochs Fine-tuned YOLO Evaluation on multiple-ingredient images

from ultralytics import YOLO
import numpy as np
import os

# Define paths
YOLO_MODEL_100_PATH = os.path.join(ROOT_DIR, 'models', 'yolo11s_100_epochs', 'weights', 'best.pt')

# Load the trained model
trained_model = YOLO(YOLO_MODEL_100_PATH)

# Evaluate the model
metrics = trained_model.val(data=MULTIPLE_INGREDIENTS_DATA_YAML_PATH)

# Extract metrics
map50 = metrics.box.map50         # mAP@0.5
map50_95 = metrics.box.map        # mAP@0.5:0.95
precision_all = metrics.box.p     # list of precision per class
recall_all = metrics.box.r        # list of recall per class
class_names = trained_model.names # class ID to name mapping

# Print mean metrics
print(f"\nOverall Metrics:")
print(f"Mean Precision: {np.mean(precision_all):.4f}")
print(f"Mean Recall: {np.mean(recall_all):.4f}")
print(f"mAP@0.5: {map50:.4f}")
print(f"mAP@0.5:0.95: {map50_95:.4f}")

# Print per-class precision and recall
print("\nPer-Class Metrics:")
for idx, (prec, rec) in enumerate(zip(precision_all, recall_all)):
    class_name = class_names[idx] if class_names and idx in class_names else f"Class {idx}"
    print(f"Class ID {idx:2d} ({class_name:<20}): Precision = {prec:.4f}, Recall = {rec:.4f}")

Ultralytics 8.3.174 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11s summary (fused): 100 layers, 9,426,732 parameters, 0 gradients, 21.4 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1824.1±370.8 MB/s, size: 109.5 KB)


val: Scanning /content/Yolov11-Ingredients-Detection/datasets/yolo_multi_ingredient_dataset/valid/labels.cache... 53 images, 0 backgrounds, 0 corrupt: 100%|██████████| 53/53 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:03<00:00,  1.18it/s]


                   all         53        169      0.444      0.353      0.378      0.254
                banana          5          8      0.508      0.625      0.659      0.532
           bell pepper          2          4      0.342        0.5      0.351      0.261
                bulgur          1          2          0          0      0.142     0.0426
               cabbage          2          3       0.95      0.333      0.501        0.1
                carrot          2          2      0.101      0.252      0.199      0.131
                cheese          3          4      0.151       0.25     0.0563     0.0507
               chicken          2          2          1          0      0.174     0.0618
             chickpeas          2          2      0.174        0.5      0.177      0.106
              cucumber          3          3      0.516          1       0.83      0.514
                   egg          6         15       0.64      0.593      0.729       0.53
              eggplan

In [ ]:
import shutil
import os

# Path to the training output directory
trained_model_dir = os.path.join(OUTPUT_DIR, 'yolo11s_100_epochs')

# Output zip file path
zip_output_path = os.path.join(OUTPUT_DIR, 'yolo11s_100_epochs.zip')

# Zip the folder
shutil.make_archive(base_name=zip_output_path.replace('.zip', ''), format='zip', root_dir=trained_model_dir)

print(f"Zipped to: {zip_output_path}")

In [ ]:
import shutil
import os

# Path to the runs directory
runs_dir = os.path.join(ROOT_DIR, 'runs')

# Output zip file path
zip_output_path = os.path.join(ROOT_DIR, 'runs.zip')

# Zip the folder
shutil.make_archive(base_name=zip_output_path.replace('.zip', ''), format='zip', root_dir=runs_dir)

print(f"Zipped to: {zip_output_path}")

Zipped to: /content/Yolov11-Ingredients-Detection/runs.zip
